In [23]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils
from utils import *

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

In [27]:
json_path = f'{GCS_FILE_PATH}vp_jsons/rt-processed_vehicle_positions_rt_2022-02-07_278_0.jsonl.gz'

In [28]:
json_path

'gs://calitp-analytics-data/data-analyses/rt_delay/vp_jsons/rt-processed_vehicle_positions_rt_2022-02-07_278_0.jsonl.gz'

In [41]:
from_json = pd.read_json(json_path, lines=True).drop(columns=['calitp_filepath'])

In [42]:
from_json

,id,vehicle,header,calitp_itp_id,calitp_url_number
0,1,"{'trip': {'tripId': '16255924', 'routeId': '20...","{'gtfsRealtimeVersion': '1.0', 'timestamp': '1...",278,0
1,2,"{'trip': {'tripId': '16189472', 'routeId': '90...","{'gtfsRealtimeVersion': '1.0', 'timestamp': '1...",278,0
2,3,"{'trip': {'tripId': '16256198', 'routeId': '20...","{'gtfsRealtimeVersion': '1.0', 'timestamp': '1...",278,0
3,4,"{'trip': {'tripId': '16254682', 'routeId': '23...","{'gtfsRealtimeVersion': '1.0', 'timestamp': '1...",278,0
4,5,"{'trip': {'tripId': '16191813', 'routeId': '70...","{'gtfsRealtimeVersion': '1.0', 'timestamp': '1...",278,0
...,...,...,...,...,...
4208,465,"{'trip': {'tripId': '16191075', 'routeId': '93...","{'gtfsRealtimeVersion': '1.0', 'timestamp': '1...",278,0
4209,466,"{'trip': {'tripId': '16191274', 'routeId': '95...","{'gtfsRealtimeVersion': '1.0', 'timestamp': '1...",278,0
4210,467,"{'trip': {'tripId': '16189208', 'routeId': '93...","{'gtfsRealtimeVersion': '1.0', 'timestamp': '1...",278,0
4211,468,"{'trip': {'tripId': '16188861', 'routeId': '1'...","{'gtfsRealtimeVersion': '1.0', 'timestamp': '1...",278,0


In [56]:
vehicle_normalized = pd.json_normalize(from_json.loc[0,'vehicle']).add_prefix('vehicle.')

In [57]:
header_normalized = pd.json_normalize(from_json.loc[0,'header']).add_prefix('header.')

In [68]:
from_json[['calitp_itp_id', 'calitp_url_number']].loc[[0]]

,calitp_itp_id,calitp_url_number
0,278,0


In [69]:
pd.concat([vehicle_normalized,
           header_normalized,
           from_json[['calitp_itp_id', 'calitp_url_number']].loc[[0]]]
          , axis=1)

,vehicle.timestamp,vehicle.trip.tripId,vehicle.trip.routeId,vehicle.position.latitude,vehicle.position.longitude,vehicle.vehicle.id,header.gtfsRealtimeVersion,header.timestamp,calitp_itp_id,calitp_url_number
0,1644278339,16255924,201,32.871883,-117.21768,1107,1.0,1644278386,278,0


In [ ]:
return_df = pd.DataFrame()

In [77]:
def normalize_vp_json(df_from_json):
    return_df = pd.DataFrame()
    for ix in df_from_json.index:
        vehicle_normalized = pd.json_normalize(df_from_json.loc[ix,'vehicle']).add_prefix('vehicle.')
        header_normalized = pd.json_normalize(df_from_json.loc[ix,'header']).add_prefix('header.')
        row = pd.concat([vehicle_normalized,
           header_normalized,
           from_json[['calitp_itp_id', 'calitp_url_number']].loc[[ix]]]
          , axis=1)
        # print(row)
        return_df = pd.concat([return_df, row])
    return return_df

In [79]:
## slow but working?
normalized_df = normalize_vp_json(from_json)

In [84]:
normalized_df = normalized_df.dropna(subset=['vehicle.timestamp']).reset_index(drop=True)

In [89]:
normalized_df['header.timestamp'].value_counts()

1644278246    469
1644278226    469
1644278386    468
1644278265    468
1644278325    468
1644278346    468
1644278285    468
1644278305    468
1644278366    467
Name: header.timestamp, dtype: int64

In [91]:
pd.json_normalize(from_json.header).timestamp.value_counts()

1644278246    469
1644278226    469
1644278386    468
1644278265    468
1644278325    468
1644278346    468
1644278285    468
1644278305    468
1644278366    467
Name: timestamp, dtype: int64

In [92]:
pd.json_normalize(from_json.vehicle).timestamp.value_counts()

1644278274    75
1644278272    72
1644278242    64
1644278269    58
1644278278    58
              ..
1644277657     2
1644278155     2
1644278142     1
1644278152     1
1644277781     1
Name: timestamp, Length: 211, dtype: int64

In [94]:
pd.json_normalize(from_json.vehicle).timestamp.max()

'1644278377'

In [95]:
pd.json_normalize(from_json.vehicle).timestamp.min()

'1644277657'

In [ ]:
## seems like not much data (because of late start?)